### XGBoost-based prediction of residual properties of aged FRP composites
Copyright (C) 2025 João Paulo D. de S. Pereira
License: GPL-3.0 (https://www.gnu.org/licenses/gpl-3.0.en.html)

In [ ]:
# Library imports
from category_encoders.target_encoder import TargetEncoder
from contextlib import suppress
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from IPython.display import display
from sklearn.metrics import r2_score, PredictionErrorDisplay
from sklearn.model_selection import train_test_split, learning_curve, KFold  
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.inspection import PartialDependenceDisplay
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import xgboost as xgb
from xgboost import plot_importance, XGBRegressor
import time
from datetime import datetime
import seaborn as sns

In [ ]:
# Record and display execution start time
start_time = time.time()
now = datetime.now()
print(now.strftime("%d/%m/%Y %H:%M:%S"))

In [ ]:
def load_dataset(
        target='Residual tensile modulus (GPa)',
        file_name='training-dataset.csv',
        min_target_counts=1,
        cols_to_drop=['Author',
            #'Type of resin',
            'Isophtalic polyester resin',
            'Orthophtalic polyester resin',
            'Vinylester resin',
            'Phenolic resin',
            'Epoxy resin',
            #'Type of fiber',
            'Glass fiber',
            'Carbon fiber',
            #'Manufacturing process',
            'Pultrusion',
            'Hand lay-up',
            'Filament winding',
            'VARTM',
            'Coupon descr.',
            'Aging effect',
            'Steady condition',
            'Cyclic condition',
            'Immersion',
            'Moisture',
            'Presence of salts',
            #'pH',
            #'Relative humidity',
            'Sustained loading',
            'Exposure time (hours)',
            'Min. exposure temperature (ºC)',
            'Max. exposure temperature (ºC)',
            #'Temperature range (ºC)',
            #'Unaged tensile modulus (GPa)',
            #'Residual tensile modulus (GPa)'
            'Tensile modulus retention'
            ]
        ):
    """Read and process a .CSV file into a filtered pandas DataFrame.

    Args:
        target (string): Column name of the target property.
        file_name (string): Name of training dataset file.
        min_target_counts (int | float): Minimum count threshold for target
            values. Rows with values appearing fewer times are dropped.
        cols_to_drop (list): List with unimportant features to be removed from
            the analysis.
    
    Returns:
        pandas.DataFrame: Filtered dataset with only relevant columns. Rows
            with values appearing fewer times are dropped.
    
    Notes:
        The .CSV file path is hardcoded in this version and not
            user-configurable. All files used herein shall be in the same folder
        XGBoost is sparsity-aware. No need to drop missing values.
    """

    # Definition of dataset path
    file_path=str(os.getcwd()) + '\\' + file_name
    
    # Read .CSV as pandas dataframe
    df = pd.read_csv(file_path, delimiter=';')

    # Drop columns that are not important for the analysis
    df = df.drop(columns=cols_to_drop)

    # Remove unnamed columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Filters target property values that appear less than min_target_count times
    value_counts = df[target].value_counts()
    valid_values = value_counts[value_counts > min_target_counts - 1].index
    df = df[df[target].isin(valid_values)]

    return df

In [ ]:
def get_abs_error(y_test, y_pred):  
    """Calculate the maximum signed absolute error between predicted and actual
        values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: Maximum signed absolute error (y_pred - y_test), rounded to
                3 decimals.
        
        Notes:
            Positive values indicate overprediction, negative underprediction.
            Both inputs must have the same length.    
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_abs_error(y_test, y_pred)
        -0.4
    """
    
    y_test = y_test.tolist()
    a = []
    for i in range(len(y_test)):
        a.append((y_pred[i] - y_test[i]))
        
    abs_error=0
    if max(abs(i) for i in a) == max(a):
        abs_error = max(a)
    else:
        abs_error = min(a)

    return round(abs_error, 3)

In [ ]:
def get_rel_error(y_test, y_pred):
    """Calculate the maximum signed relative error between predicted and actual
        values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: Maximum signed relative error ((y_pred - y_test) / y_test),
                rounded to 3 decimals.
        
        Notes:
            Positive values indicate overprediction, negative underprediction.
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_rel_error(y_test, y_pred)
        -0.167
    """
    
    y_test = y_test.tolist()
    a = []
    for i in range(len(y_test)):
        a.append((y_pred[i] - y_test[i])/y_test[i])
        
    rel_error=0
    if max(abs(i) for i in a) == max(a):
        rel_error = max(a)
    else:
        rel_error = min(a)

    return round(rel_error, 3)

In [ ]:
def get_mae(y_test, y_pred):
    """Calculate the mean absolute error (MAE) between predicted and actual values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: MAE (Σ(|y_pred - y_test|) / len(y_test)), rounded to 3 decimals.
        
        Note:
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_mae(y_test, y_pred)
        0.233
    """
    
    y_test = y_test.tolist()
    a = 0
    for i in range(len(y_test)):
        a = a + abs(y_pred[i] - y_test[i])
        
    mae = a / len(y_test)

    return round(mae, 3)

In [ ]:
def get_rmse(y_test, y_pred):
    """Calculate the root mean squared error (RMSE) between predicted and
        actual values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: RMSE ((Σ(|y_pred - y_test|)² / len(y_test))^0.5), rounded
                to 3 decimals.
        
        Note:
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_rmse(y_test, y_pred)
        0.265
    """
    
    y_test = y_test.tolist()
    a = 0
    for i in range(len(y_test)):
        a = a + (y_pred[i] - y_test[i]) ** 2
        
    rmse = np.sqrt(a / len(y_test))

    return round(rmse, 3)

In [ ]:
def get_r2(y_test, y_pred):
    """Calculate the coefficient of determination (or R² score) of predicted
        values.
    
    Args:
        y_test (pandas.DataFrame): Actual values.
        y_pred (numpy.ndarray): Predicted values from the model.
    
        Returns:
            float: R² score (1 - RSS/TSS), rounded to 3 decimals.
        
        Note:
            RSS: Residual sum of squares (Σ(y_pred - y_test)²).
            TSS: Total sum of squares (Σ(mean - y_test)²), i.e., variability of
                the actual data.
            An R² score equal to 1 indicates a perfect fit between predicted and
                actual values. An R² equal to 0 means the prediction is no better
                than the mean. A negative R² indicates the prediction is worse
                than the mean.
            Both inputs must have the same length.
        
        Example:
        \\>>> y_test = pd.DataFrame([1.2, 3.4, 5.6])
        \\>>> y_pred = np.array([1.0, 3.5, 5.2])
        \\>>> get_r2(y_test, y_pred)
        0.978
    """
    
    y_test = y_test.tolist()
    mean = sum(y_test) / len(y_test)
    rss = 0  # Residual sum of squares
    tss = 0  # Total sum of squares
    for i in range(len(y_test)):
        rss = rss + (y_pred[i] - y_test[i]) ** 2
        tss = tss + (mean - y_test[i]) ** 2

    r2 = 1 - rss / tss

    return round(r2, 3)

In [ ]:
class Predictor:
    """Train XGBoost model with Bayesian optimization and predict the long-term
        property retention of aged FRP composites.
    """

    def __init__(
        self,
        model_params: dict = {
            'encoder': TargetEncoder(),
            'clf': {
                'random_state'  :0,
                'booster'       :'gbtree',
                'base_score'    :0.9,
                'eval_metric'   :r2_score
                },
            'bayes_cv': {
                'search_space': {
                    'clf__max_depth'            :Integer(7, 10),
                    'clf__n_estimators'         :Integer(500, 1000),
                    'clf__learning_rate'        :Real(0.05, 0.1,
                        prior='log-uniform'),
                    'clf__subsample'            :Real(0.8, 1.0),
                    'clf__colsample_bytree'     :Real(0.8, 1.0),
                    'clf__reg_alpha'            :Real(0.1, 0.5),
                    'clf__reg_lambda'           :Real(0.1, 0.5),
                    'clf__gamma'                :Real(0.0, 0.05),
                    'clf__min_child_weight'     :Integer(1, 2)
                    },
                'cv'            :5,
                'n_iter'        :60,
                'scoring'       :'r2',
                'n_jobs'        :6
                }
            },
            target='Residual tensile modulus (GPa)'
        ):
        """Args:
            model_params (dict, optional): Dictionary containing all model parameters.
                Structure:
                - encoder: Categorical variable encoder.
                - clf: Estimator parameters.
                    - random_state (int): Seed for reproducibility.
                    - booster (str): Booster type.
                    - base_score (float): Initial prediction value.
                    - eval_metric  (Union[str, List[str], Callable, NoneType]):
                        Metric for training result monitoring.
                - bayes_cv: Bayesian optimization parameters (hyperparameter tuning).
                    - search_space: Hyperparameter ranges for optimization.
                    - cv: Number of cross-validation folds.
                    - n_iter: Number of hyperparameter combinations to test.
                    - n_jobs: Number of CPU cores to be used in parallel.
            target (str, optional): Target property name.

        Attributes:
            - model_params (dict): Stored model parameters.
            - target (str): Stored target property.
            - cv_strategy (KFold): Cross-validation configuration. Shuffling is
                enabled to ensure representative samples in each split.
            - dataset (dict): Will store dataset information.
            - n_runs (int): Will store number of models to be trained.
            - trained_models (list): Will store models and metrics dicts after training.
            - overall_stats (dict): Will store statiscally treated errors and R² score.
            - best_model (XGBRegressor): Will store top-performing model.
        """

        self.model_params   = model_params
        self.target         = target
        self.cv_strategy    = KFold(n_splits=5, shuffle=True)#, random_state=0)
        self.dataset        = {
            'dataset'           :0,
            'X'                 :0,
            'y'                 :0
        }
        self.n_runs         = 0
        self.trained_models = []
        self.overall_stats  = {
            'abs_error_mean'    :0,
            'abs_error_std'     :0,
            'rel_error_mean'    :0,
            'rel_error_std'     :0,
            'mae_mean'          :0,
            'mae_std'           :0,
            'rmse_mean'         :0,
            'rmse_std'          :0,
            'r2_mean'           :0,
            'r2_std'            :0
        }
        self.best_model     = None
        
    def create_model(self, log_transf=[]):
        """Sets up a Bayesian-optimized XGBoost regression pipeline including:
            - Optional log-transformation of specified features
            - Data preprocessing (encoding)
            - XGBoost regressor with Bayesian hyperparameter optimization
        
        Args:
            log_transf (list, optional): List of feature names to apply log10
                transformation. Useful for handling skewed distributions and
                outliers. Defaults to empty list.
                
        Returns:
            skopt.BayesSearchCV: Configured Bayesian optimization search object with:
                - Preprocessing pipeline (encoder + XGBoost)
                - Parameter search space
                - Cross-validation strategy
                - Evaluation metrics
                
        Notes:
            - Pipeline includes automatic encoding of categorical variables
            - Uses XGBoost's 'reg:squarederror' objective by default
        """
        
        # Load dataset
        df = load_dataset(target='Residual tensile modulus (GPa)')

        # Log-transform features with outlier values
        for i in range(len(log_transf)):
            df[log_transf[i]] = np.log10(df[log_transf[i]])
        
        # Separate independent variables from target
        X = df.drop(columns=self.target)
        y = df[self.target]
        
        # Store dataset and splits
        self.dataset['dataset'] = df
        self.dataset['X'] = X
        self.dataset['y'] = y

        # Configure training pipeline and hyperparameter tuning
        estimators = [
            ('encoder', self.model_params['encoder']),
            ('clf', XGBRegressor(
                #random_state            =self.model_params['clf']['random_state'],
                booster                 =self.model_params['clf']['booster'],
                base_score              =self.model_params['clf']['base_score'],
                eval_metric             =self.model_params['clf']['eval_metric'],
                objective               ='reg:squarederror'
                )
            )
        ]
                
        opt = BayesSearchCV(
            Pipeline(steps=estimators),
            self.model_params['bayes_cv']['search_space'],
            cv                  = self.cv_strategy,
            n_iter              = self.model_params['bayes_cv']['n_iter'],
            scoring             = self.model_params['bayes_cv']['scoring'],
            error_score         = 'raise',
            random_state        = 0,
            return_train_score  = True,
            n_jobs              = self.model_params['bayes_cv']['n_jobs'],
            n_points            = 1,
            verbose             = False
            )
        
        return opt

    def train(self, n_runs=10, q=5, test_size=0.2, best_metric='r2', log_transf=[]):
        """Performs training pipeline for n_runs models including:
            - Dataset preparation with optional log transformations
            - Multiple train-test splits with target-stratified sampling
            - Bayesian-optimized XGBoost training for each split
            - Performance evaluation (abs. error, rel. error, MAE, RMSE and R²)

        Args:
            n_runs (int, optional): Number of independent training runs with
                different data splits. Defaults to 10.
            q (int, optional): Number of quantiles for stratified sampling of
                continuous target variable. Defaults to 5.
            test_size (float, optional): Proportion of dataset to use for
                validation. Defaults to 0.2.
            best_metric (str, optional): Metric used to select best model.
                Options: 'r2' (R-squared), 'mae' (Mean Absolute Error), 'rmse'
                (Root Mean Squared Error). Defaults to 'r2'.
            log_transf (list, optional): List of feature names to apply log10
                transformation. Defaults to empty list.

        Returns:
            None: Updates object attributes with trained models and statistics.

        Attributes Modified:
            trained_models (list): Stores dictionaries containing:
                - Trained model objects
                - Training/test data
                - Prediction results
                - Evaluation metrics
                - Feature importance
            overall_stats (dict): Aggregated statistics across all runs:
                - Absolute/relative errors
                - Mean ± std of MAE, RMSE, R²
            best_model (dict): Information for best-performing model based on
                best_metric

        Notes:
            - Uses quantile-based stratification to maintain target distribution
            - Implements Bayesian optimization for hyperparameter tuning
            - Stores model information for post-analysis
        """

        # Create model
        model = self.create_model(log_transf=log_transf)

        # Load dataset
        df = self.dataset['dataset']
        X = df.drop(columns=self.target)
        y = df[self.target]
        
        # Restart attributes
        self.n_runs = n_runs
        self.trained_models = []

        # Bin the continuous target into categories for stratification
        bins = pd.qcut(y, q=q, duplicates='drop')

        loop_start = time.time()
        for i in range(n_runs):

            # Split data in training and test groups
            X_train, X_test, y_train, y_test = train_test_split(X, y,
                test_size=test_size, stratify=bins
                )
        
            # Train model
            print('Starting fit #', i+1, sep='')
            fit_start = time.time()
            print(datetime.now().strftime("%d/%m/%Y %H:%M:%S\n"))
            model.fit(X_train, y_train)
            fit_finish = time.time()
            print('Fit #', i+1, ' is finished! (%s s)\n'
                % round((fit_finish - fit_start), 1), sep='')
            if i == n_runs - 1:
                print('Total training duration: %s s\n'
                    % round((fit_finish - loop_start), 1), sep='')
            y_pred = model.predict(X_test)
            model_info = {
                'id': i,
                'model': model.best_estimator_,
                'X_train': X_train,
                'X_test': X_test,
                'y_train': y_train,
                'y_pred': y_pred,
                'y_test': y_test,
                'abs_error': get_abs_error(y_test, y_pred),
                'rel_error': get_rel_error(y_test, y_pred),
                'mae': get_mae(y_test, y_pred),
                'rmse': get_rmse(y_test, y_pred),
                'r2': get_r2(y_test, y_pred),
                'best_cv_score': model.best_score_,
                'best_params': model.best_params_,
                'importance_plot': model.best_estimator_.steps[1][1],
                'importance_score': model.best_estimator_.steps[1][1].
                    get_booster().get_score(importance_type='weight')
                }
            self.trained_models.append(model_info)
        
        # Extract metrics, calculate and store statistics
        abs_error = [self.trained_models[i]['abs_error']
            for i in range(len(self.trained_models))]
        abs_error_mean = np.mean(abs_error)
        abs_error_std = np.std(abs_error)
        self.overall_stats['abs_error_mean'] = abs_error_mean
        self.overall_stats['abs_error_std'] = abs_error_std
        print(f"Abs. error: {abs_error_mean:.3f} ± {abs_error_std:.3f}")

        rel_error = [self.trained_models[i]['rel_error']
            for i in range(len(self.trained_models))]
        rel_error_mean = np.mean(rel_error)
        rel_error_std = np.std(rel_error)
        self.overall_stats['rel_error_mean'] = rel_error_mean
        self.overall_stats['rel_error_std'] = rel_error_std
        print(f"Rel. error: {rel_error_mean:.3f} ± {rel_error_std:.3f}")

        mae = [self.trained_models[i]['mae']
            for i in range(len(self.trained_models))]
        mae_mean = np.mean(mae)
        mae_std = np.std(mae)
        self.overall_stats['mae_mean'] = mae_mean
        self.overall_stats['mae_std'] = mae_std
        print(f"MAE: {mae_mean:.3f} ± {mae_std:.3f}")

        rmse = [self.trained_models[i]['rmse']
            for i in range(len(self.trained_models))]
        rmse_mean = np.mean(rmse)
        rmse_std = np.std(rmse)
        self.overall_stats['rmse_mean'] = rmse_mean
        self.overall_stats['rmse_std'] = rmse_std
        print(f"RMSE: {rmse_mean:.3f} ± {rmse_std:.3f}")

        r2 = [self.trained_models[i]['r2']
            for i in range(len(self.trained_models))]
        r2_mean = np.mean(r2)
        r2_std = np.std(r2)
        self.overall_stats['r2_mean'] = r2_mean
        self.overall_stats['r2_std'] = r2_std
        print(f"R²: {r2_mean:.2f} ± {r2_std:.2f}\n")
    
        # Store the best model
        best_model = max(self.trained_models, key=lambda x: x['r2'])
        self.best_model = best_model
    
    def plot_learning_curves(self, q=5, test_size=0.2, metrics=[1,1,1]):
        """...

        Args:
            ...
            metrics (list): List with metrics to be plotted: MAE, RMSE, and R²,
                respectively. 1 is plot, 0 is not plot. Default is to plot only R²
        """

        """Plots MAE, RMSE, and R² learning curves to assess underfitting/overfitting
        
        Args:
            q (int, optional): Number of quantiles for stratification.
                Defaults to 5.
            test_size (float, optional): Proportion of dataset to use for
                validation. Defaults to 0.2.
            metrics (list, optional): Which metrics to plot [MAE, RMSE, R²]. 
                1=plot, 0=skip. Defaults to [1,1,1] (all metrics).

        Returns:
            None: Displays matplotlib figures with learning curves and prints
                mean ± std across CV folds.
                
        Notes:
            - Uses stratified sampling to maintain target distribution
        """
        
        # Separate independent variables from target
        X = self.dataset['X']
        y = self.dataset['y']

        # Bin the continuous target into categories for stratification
        bins = pd.qcut(y, q=q, duplicates='drop')

        # Split into train and test data
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=bins, test_size=test_size)

        # Define marker styles for different models
        markers = ['o', 's', '^', 'D', 'v', 'p', '*', 'h', 'X', 'd']  # Add more if needed
        fill = ['full', 'none']  # Alternate filled/unfilled

        if metrics[0] == 1:
            plt.figure(figsize=(10, 6))
            
            train_score_total = []
            test_score_total = []

            for i in range(len(self.trained_models)):
                train_sizes, train_scores, test_scores, fit_times, score_times = learning_curve(  
                    self.trained_models[i]['model'], X, y, train_sizes=np.linspace(0.1, 1.0, 8),
                    cv=self.cv_strategy, scoring='r2', return_times=True
                )

                # Converting MAE ro positive
                train_scores = -train_scores
                test_scores = -test_scores

                print(f'\nModel {i+1}')

                d = []
                for j, (a, b, c) in enumerate(zip(train_sizes, train_scores, test_scores)):
                    # print(f"CV {j+1}: {a} samples, train MAE = {b.mean():.3f}, valid MAE = {c.mean():.3f} ({sum(fit_times[j]):.2f} s)")
                    d.append(sum(fit_times[j]))
                    if j == len(train_sizes) - 1:
                        train_score_total.append(b)
                        test_score_total.append(c)
                # print(f"{sum(d):.0f} s in total")

                # Calculate training and validation means and stds
                train_mean = np.mean(train_scores, axis=1)
                train_std = np.std(train_scores, axis=1)
                test_mean = np.mean(test_scores, axis=1)
                test_std = np.std(test_scores, axis=1)

                # Define marker and fill styles
                marker = markers[i // 2 % len(markers)]  # Change marker every 2 models
                fillstyle = fill[i % 2]  # Alternate fill for each model

                # Plot training curve
                plt.plot(
                    train_sizes,
                    train_mean,
                    marker=marker,
                    fillstyle=fillstyle,
                    color='blue',
                    linestyle='-',
                    alpha=0.4,
                    label=f'Train' if i == 0 else ''
                    )
                plt.fill_between(
                    train_sizes,
                    train_mean - train_std,
                    train_mean + train_std,
                    alpha=0.05,
                    color='blue'
                    )

                # Plot validation curve
                plt.plot(
                    train_sizes,
                    test_mean,
                    marker=marker,
                    color='red',
                    linestyle='-',
                    alpha=0.4,
                    label=f'Validation' if i == 0 else ''
                    )
                plt.fill_between(
                    train_sizes,
                    test_mean - test_std,
                    test_mean + test_std,
                    alpha=0.05,
                    color='red'
                    )
                
            print(f"Train MAE: {np.mean(train_score_total):.3f} ± {np.std(train_score_total):.3f}")
            print(f"Valid MAE: {np.mean(test_score_total):.3f} ± {np.std(test_score_total):.3f}")

            # Plot formatting
            plt.xlabel("Training set size")
            plt.ylabel("MAE")
            plt.ylim(-1.1, -0.4)  # Adjust based on your scores
            plt.title('Learning curves across models')
            plt.grid(True)

            # Create custom legend
            legend_elements = []
            legend_elements.extend([
                Line2D([0], [0], color='blue', lw=2, label='Training'),
                Line2D([0], [0], color='red', lw=2, label='Validation')
            ])

            for i in range(len(self.trained_models)):
                marker_idx = i // 2 % len(markers)
                fill_idx = i % 2
                legend_elements.append(Line2D(
                    [0], [0],
                    marker=markers[marker_idx],
                    fillstyle=fill[fill_idx],
                    color='gray',
                    label=f'Model {i+1}',
                    linestyle='None',
                    markersize=10
                ))

            plt.legend(handles=legend_elements, ncol=3, loc='upper right')

            plt.tight_layout()
            plt.show()


        if metrics[1] == 1:
            plt.figure(figsize=(10, 6))

            train_score_total = []
            test_score_total = []

            for i in range(len(self.trained_models)):
                train_sizes, train_scores, test_scores, fit_times, score_times = learning_curve(  
                    self.trained_models[i]['model'], X, y, train_sizes=np.linspace(0.1, 1.0, 8),
                    cv=self.cv_strategy, scoring='neg_root_mean_squared_error', return_times=True
                )

                # Converting MAE ro positive
                train_scores = -train_scores
                test_scores = -test_scores

                print(f'\nModel {i+1}')

                d = []
                for j, (a, b, c) in enumerate(zip(train_sizes, train_scores, test_scores)):
                    # print(f"CV {j+1}: {a} samples, train RMSE = {b.mean():.3f}, valid RMSE = {c.mean():.3f} ({sum(fit_times[j]):.2f} s)")
                    d.append(sum(fit_times[j]))
                    if j == len(train_sizes) - 1:
                        train_score_total.append(b)
                        test_score_total.append(c)
                # print(f"{sum(d):.0f} s in total")

                # Calculate training and validation means and stds
                train_mean = np.mean(train_scores, axis=1)
                train_std = np.std(train_scores, axis=1)
                test_mean = np.mean(test_scores, axis=1)
                test_std = np.std(test_scores, axis=1)

                # Define marker and fill styles
                marker = markers[i // 2 % len(markers)]  # Change marker every 2 models
                fillstyle = fill[i % 2]  # Alternate fill for each model

                # Plot training curve
                plt.plot(
                    train_sizes,
                    train_mean,
                    marker=marker,
                    fillstyle=fillstyle,
                    color='blue',
                    linestyle='-',
                    alpha=0.4,
                    label=f'Train' if i == 0 else ''
                    )
                plt.fill_between(
                    train_sizes,
                    train_mean - train_std,
                    train_mean + train_std,
                    alpha=0.05,
                    color='blue'
                    )

                # Plot validation curve
                plt.plot(
                    train_sizes,
                    test_mean,
                    marker=marker,
                    color='red',
                    linestyle='-',
                    alpha=0.4,
                    label=f'Validation' if i == 0 else ''
                    )
                plt.fill_between(
                    train_sizes,
                    test_mean - test_std,
                    test_mean + test_std,
                    alpha=0.05,
                    color='red'
                    )
            
            print(f"Train RMSE: {np.mean(train_score_total):.3f} ± {np.std(train_score_total):.3f}")
            print(f"Valid RMSE: {np.mean(test_score_total):.3f} ± {np.std(test_score_total):.3f}")

            # Plot formatting
            plt.xlabel("Training set size")
            plt.ylabel("RMSE")
            plt.ylim(0, 7)  # Adjust based on your scores
            plt.title('Learning curves across models')
            plt.grid(True)

            # Create custom legend
            legend_elements = []
            legend_elements.extend([
                Line2D([0], [0], color='blue', lw=2, label='Training'),
                Line2D([0], [0], color='red', lw=2, label='Validation')
            ])

            for i in range(len(self.trained_models)):
                marker_idx = i // 2 % len(markers)
                fill_idx = i % 2
                legend_elements.append(Line2D(
                    [0], [0],
                    marker=markers[marker_idx],
                    fillstyle=fill[fill_idx],
                    color='gray',
                    label=f'Model {i+1}',
                    linestyle='None',
                    markersize=10
                ))

            plt.legend(handles=legend_elements, ncol=3, loc='upper right')

            plt.tight_layout()
            plt.show()


        if metrics[2] == 1:
            plt.figure(figsize=(10, 6))

            train_score_total = []
            test_score_total = []
        
            for i in range(len(self.trained_models)):
                train_sizes, train_scores, test_scores, fit_times, score_times = learning_curve(  
                    self.trained_models[i]['model'], X, y, train_sizes=np.linspace(0.1, 1.0, 8),
                    cv=self.cv_strategy, scoring='r2', return_times=True
                )

                print(f'\nModel {i+1}')

                d =[]
                for j, (a, b, c) in enumerate(zip(train_sizes, train_scores, test_scores)):
                    # print(f"CV {j+1}: {a} samples, train R² = {b.mean():.2f}, valid R² = {c.mean():.2f} ({sum(fit_times[j]):.2f} s)")
                    d.append(sum(fit_times[j]))
                    if j == len(train_sizes) - 1:
                        train_score_total.append(b)
                        test_score_total.append(c)
                # print(f"{sum(d):.0f} s in total")

                # Calculate training and validation means and stds
                train_mean = np.mean(train_scores, axis=1)
                train_std = np.std(train_scores, axis=1)
                test_mean = np.mean(test_scores, axis=1)
                test_std = np.std(test_scores, axis=1)

                # Define marker and fill styles
                marker = markers[i // 2 % len(markers)]  # Change marker every 2 models
                fillstyle = fill[i % 2]  # Alternate fill for each model

                # Plot training curve
                plt.plot(
                    train_sizes,
                    train_mean,
                    marker=marker,
                    fillstyle=fillstyle,
                    color='blue',
                    linestyle='-',
                    alpha=0.4,
                    label=f'Train' if i == 0 else ''
                    )
                plt.fill_between(
                    train_sizes,
                    train_mean - train_std,
                    train_mean + train_std,
                    alpha=0.05,
                    color='blue'
                    )

                # Plot validation curve
                plt.plot(
                    train_sizes,
                    test_mean,
                    marker=marker,
                    color='red',
                    linestyle='-',
                    alpha=0.4,
                    label=f'Validation' if i == 0 else ''
                    )
                plt.fill_between(
                    train_sizes,
                    test_mean - test_std,
                    test_mean + test_std,
                    alpha=0.05,
                    color='red'
                    )
                    
            print(f"Train R²: {np.mean(train_score_total):.2f} ± {np.std(train_score_total):.2f}")
            print(f"Valid R²: {np.mean(test_score_total):.2f} ± {np.std(test_score_total):.2f}")
            
            # Plot formatting
            plt.xlabel("Training set size")
            plt.ylabel("R² score")
            plt.ylim(0.4, 1.1)  # Adjust based on your scores
            plt.title('Learning curves across models')
            plt.grid(True)

            # Create custom legend
            legend_elements = []
            legend_elements.extend([
                Line2D([0], [0], color='blue', lw=2, label='Training'),
                Line2D([0], [0], color='red', lw=2, label='Validation')
            ])

            for i in range(len(self.trained_models)):
                marker_idx = i // 2 % len(markers)
                fill_idx = i % 2
                legend_elements.append(Line2D(
                    [0], [0],
                    marker=markers[marker_idx],
                    fillstyle=fill[fill_idx],
                    color='gray',
                    label=f'Model {i+1}',
                    linestyle='None',
                    markersize=10
                ))

            plt.legend(handles=legend_elements, ncol=3, loc='lower right')

            plt.tight_layout()
            plt.show()

    def plot_feature_distribution(self, main_feature='Exposure time (days)'):
        """Generates four diagnostic plots:
            1. Boxplot of target variable distribution
            2. Pairplot of all feature distributions
            3. Scatter plot of main feature vs target
            4. Value counts of main feature
        
        Args:
            main_feature (str, optional): Feature to analyze in detail.
                Defaults to 'Exposure time (days)'.
        
        Returns:
            None: Displays matplotlib figures with feature distribution plots.
        """
        
        df = self.dataset['dataset']
        
        print('\nTarget boxplot distribution')
        sns.boxplot(x=self.dataset['y'])  # Check target
        plt.show()

        print('\nOverall feature distribution')
        sns.pairplot(df)  # Check feature distributions
        # sns.pairplot(df, diag_kind='kde')  # Unncomment for smoothed curves
        plt.show()

        #  Outlier identification for most important feature
        print('\nMost important feature vs. target')
        sns.scatterplot(x=main_feature, y=self.target, data=df)
        plt.show()

        # Check individual feature distribution
        print('\nMost important feature distribution')
        display(self.dataset['X'][main_feature].value_counts())

    def plot_pred_error(self):
        """Generate prediction error plots for all trained models.

        Returns:
            None: Displays matplotlib figures with error plots.
    
        Notes:
            - Points on diagonal line indicate perfect predictions
            - Systematic deviations suggest model bias
        """
        
        for i in range(len(self.trained_models)):
            print(f'\nModel {i+1}')
            display = PredictionErrorDisplay.from_predictions(
                self.trained_models[i]['y_test'],
                self.trained_models[i]['y_pred'],
                kind="actual_vs_predicted")
            display.plot()
            plt.show()
    
    def plot_importance(self):
        """Visualize feature importance scores for all trained models.
        
        Returns:
            None: Displays matplotlib figures with feature distribution plots.

        Notes:
            - Importance types available: ['weight', 'gain', 'cover', 'total_gain',
                'total_cover']
        """
        
        for i in range(len(self.trained_models)):
            print(f'\nModel {i+1}')
            self.trained_models[i]['importance_score']
            with plt.style.context("ggplot"):
                fig = plt.figure(figsize=(6,6))
                ax = fig.add_subplot(111)
                xgb.plotting.plot_importance(self.trained_models[i]['importance_plot'], ax=ax)
            plt.show()

    def plot_tree(self, n_model=0, n_tree=0):
        """Visualize individual decision tree from XGBoost model.
    
        Args:
            n_model (int, optional): Model index to visualize. Defaults to 0.
            n_tree (int, optional): Tree number to visualize. Defaults to 0.

        Returns:
            None: Displays matplotlib figure with decision tree.
        """
        
        with plt.style.context("ggplot"):
            fig = plt.figure(figsize=(25,10))
            ax = fig.add_subplot(111)
            xgb.plotting.plot_tree(self.trained_models[n_model]['importance_plot'], ax=ax, num_trees=n_tree)

    def residual_analysis(self, feature):
        """Generates four diagnostic plots for each trained model to assess prediction quality:
            1. Residual distribution histogram with kernel density estimation (KDE)
            2. Residuals vs. actual values scatter plot
            3. Residuals vs. specified feature scatter plot
            4. Q-Q plot for normality assessment

        Args:
            feature (str): Name of feature to analyze (must match column
                name in dataset).
        
        Returns:
            None: Displays matplotlib figures with residual analysis plots and
                prints 95% prediction interval (±1.96σ) for each model.

        Notes:
            - Red dashed line in scatter plots indicates perfect prediction
                (residual = 0).
            - Q-Q plot should follow straight line for normally distributed
                residuals.
            - Heteroscedasticity (changing variance) appears as funnel shape in
                residual plots.
        """

        for i in range(len(self.trained_models)):
            y_test = self.trained_models[i]['y_test']
            y_pred = self.trained_models[i]['y_pred']
            X_test = self.trained_models[i]['X_test']

            residuals = y_test - y_pred
            std_dev = residuals.std()
            
            print(f"95% prediction interval: ±{1.96*std_dev:.2f} GPa")

            # 1. Residual Distribution Plot
            plt.figure(figsize=(10, 4))
            sns.histplot(residuals, kde=True, bins=30)
            plt.title("Residual Distribution")
            plt.xlabel("Prediction Error (GPa)")
            plt.show()

            # 2. Residual vs. Actual Plot
            plt.figure(figsize=(10, 4))
            plt.scatter(y_test, residuals, alpha=0.3)
            plt.axhline(0, color='r', linestyle='--')
            plt.title("Residuals vs Actual Values")
            plt.xlabel("Actual Residual Modulus (GPa)")
            plt.ylabel("Prediction Error (GPa)")
            plt.show()

            # 3. Residual vs. Key Features
            plt.figure(figsize=(10, 4))
            plt.scatter(X_test['Exposure time (days)'], residuals, alpha=0.3)
            plt.axhline(0, color='r', linestyle='--')
            plt.title("Residuals vs Exposure Time")
            plt.xlabel("Exposure Time (days)")
            plt.ylabel("Prediction Error (GPa)")
            plt.show()

            from scipy import stats
            stats.probplot(residuals, dist="norm", plot=plt)
            plt.title("Q-Q Plot")
            plt.show()
    
    def lin_correl(self):
        """Calculate and display linear correlations (Pearson correlation coefficients)
        between numerical features and target.

        Returns:
            None: Prints linear correlation of features in relation to target.
        
        Notes:
            - Only considers numerical features (automatically excludes categoricals)
            - Pearson correlation measures linear relationships (-1 to +1)
            - Values near 0 indicate no linear correlation
        """

        X = self.dataset['X']
        y = self.dataset['y']

        # Calculate linear correlations
        corr = pd.DataFrame(X).select_dtypes(include=['number']).corrwith(pd.Series(y))
        print(corr.sort_values(ascending=False))

    def pdp(self, i=0, feature=['Exposure time (days)']):
        """Plot partial dependence plots (PDP) for specified features in relation
        to target while the influence from other features is marginalized.
        
        Args:
            i (int): Index of training model to be used. Defaults to 0.
            feature (list): List of feature names to analyze.

        Returns:
            None: Displays matplotlib figures with PDP.
        """

        X = self.dataset['X']

        PartialDependenceDisplay.from_estimator(self.trained_models[i]['model'], X, features=[feature])
        plt.show()

    def holdout_test(self, file_name):
        """Evaluate models on holdout data to assess the model's capability to
            extrapolate features beyond training data ranges.
    
        Args:
            file_name (str): Name of holdout dataset file.
            
        Todo:
            - Implement prediction and evaluation on holdout set
            - Add metrics comparison to training performance
        """

        for i in range(len(self.trained_models)):
            self.predict()
    
    def bootstrap(self):
        """Calculate bootstrap confidence intervals for model metrics. Estimates
            uncertainty in model performance by resampling with replacement.
        
        Returns:
            dict: Dictionary containing lists of bootstrapped metrics:
                - 'mae': Mean absolute error values
                - 'rmse': Root mean squared error values 
                - 'r2': R² values
                
        Todo:
            - Implement bootstrap resampling procedure
            - Calculate confidence intervals
        """

        metrics = {
            'mae': [],
            'rmse': [],
            'r2': []
        }

        model = self.best_model

    def predict(self, model):
        pass

In [ ]:
predictor = Predictor(target='Residual tensile modulus (GPa)')
predictor.train(n_runs=4, log_transf=['Exposure time (days)'])

In [ ]:
predictor.plot_learning_curves(metrics=[1,1,1])

In [ ]:
predictor.plot_feature_distribution(main_feature='Exposure time (days)')

In [ ]:
predictor.plot_pred_error()

In [ ]:
predictor.plot_importance()

In [ ]:
predictor.plot_tree()

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

df = load_dataset(target='Residual tensile modulus (GPa)')
X = df.drop(columns=predictor.target)
y = df[predictor.target]

# Outer CV: Evaluate performance
for i in range(len(predictor.trained_models)):
    outer_scores = cross_val_score(
        predictor.trained_models[i]['model'], X, y, 
        cv=KFold(5, shuffle=True), 
        scoring='r2'
    )
    print(f"Unbiased R²: {np.mean(outer_scores):.3f} ± {np.std(outer_scores):.3f}")

In [ ]:
predictor.lin_correl()

In [ ]:
predictor.residual_analysis()

In [ ]:
# Fim do processamento
now = datetime.now()

print(now.strftime("%d/%m/%Y %H:%M:%S"))
print("\nProcessing time: %s seconds\n" % (round((time.time() - start_time), 2)))